In [1]:
# --- Cellule 1 : Dépendances (à exécuter une seule fois) ---
# !pip -q install "pymupdf>=1.24.9" "transformers>=4.43" "sentencepiece>=0.1.99" "accelerate>=0.33" bitsandbytes sacremoses -U

In [ ]:
SOURCE_PDF = "/Users/gauthier/Documents/Travail/Projets/IFG/TASK HUB/Le cadre théorique/04 Sources essentielles de TASK™/9789240074323-eng.pdf"

In [3]:
from transformers import MarianTokenizer, MarianMTModel
MODEL_NAME = "Helsinki-NLP/opus-mt-en-fr"
MAX_NEW_TOKENS = 512                           # limite sortie par ligne
BATCH_SIZE = 16                                # taille lot traduction
DEVICE = "cpu"


tokenizer  = MarianTokenizer.from_pretrained(MODEL_NAME)
model  = MarianMTModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()

In [4]:
import fitz  # PyMuPDF

doc = fitz.open(SOURCE_PDF)

# Extraction des blocs de texte
pages_text = []
for page in doc:
    blocks = page.get_text("blocks")  # [(x0,y0,x1,y1,"texte",bloc_num,...)]
    page_blocks = []
    for b in blocks:
        text = b[4].strip()
        if len(text) >= 4:  # on ignore les bouts trop courts
            page_blocks.append((b[:4], text))  # coordonnées + texte
    pages_text.append(page_blocks)

print("Exemple de blocs :", pages_text[0][:3])

Exemple de blocs : [((215.73899841308594, 773.1566772460938, 253.12347412109375, 799.91845703125), 'HEALTH\nFOR ALL'), ((185.19700622558594, 266.8390197753906, 401.7681579589844, 468.8710021972656), 'World \nHealth \nStatistics \n2023'), ((336.1419982910156, 563.7219848632812, 551.9020385742188, 687.9960327148438), 'Monitoring \nhealth for the \nSDGs\nSustainable Development Goals')]


In [5]:
def translate_texts(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    translated = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

translated_pages = []
for page_idx, page_blocks in enumerate(pages_text, start=1):
    print(f"\n=== Traitement de la page {page_idx}/{len(pages_text)} ===")
    
    texts = [t for _, t in page_blocks]
    translations = []
    
    for i in range(0, len(texts), BATCH_SIZE):
        batch = texts[i:i+BATCH_SIZE]
        print(f"  - Traduction des blocks {i+1} à {min(i+BATCH_SIZE, len(texts))} sur {len(texts)}")
        
        batch_translations = translate_texts(batch)
        translations.extend(batch_translations)
    
    page_translated = [(coords, trans) for (coords, _), trans in zip(page_blocks, translations)]
    translated_pages.append(page_translated)
    
    print(f"  ✅ Page {page_idx} terminée ({len(page_blocks)} blocks traduits)")

print("\nExemple traduction première page :", translated_pages[0][:3])


=== Traitement de la page 1/136 ===
  - Traduction des blocks 1 à 3 sur 3
  ✅ Page 1 terminée (3 blocks traduits)

=== Traitement de la page 2/136 ===
  ✅ Page 2 terminée (0 blocks traduits)

=== Traitement de la page 3/136 ===
  - Traduction des blocks 1 à 3 sur 3
  ✅ Page 3 terminée (3 blocks traduits)

=== Traitement de la page 4/136 ===
  - Traduction des blocks 1 à 16 sur 16
  ✅ Page 4 terminée (16 blocks traduits)

=== Traitement de la page 5/136 ===
  - Traduction des blocks 1 à 16 sur 24
  - Traduction des blocks 17 à 24 sur 24
  ✅ Page 5 terminée (24 blocks traduits)

=== Traitement de la page 6/136 ===
  ✅ Page 6 terminée (0 blocks traduits)

=== Traitement de la page 7/136 ===
  - Traduction des blocks 1 à 9 sur 9
  ✅ Page 7 terminée (9 blocks traduits)

=== Traitement de la page 8/136 ===
  - Traduction des blocks 1 à 16 sur 56
  - Traduction des blocks 17 à 32 sur 56
  - Traduction des blocks 33 à 48 sur 56
  - Traduction des blocks 49 à 56 sur 56
  ✅ Page 8 terminée (56 

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (512). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


  - Traduction des blocks 17 à 23 sur 23
  ✅ Page 103 terminée (23 blocks traduits)

=== Traitement de la page 104/136 ===
  - Traduction des blocks 1 à 15 sur 15
  ✅ Page 104 terminée (15 blocks traduits)

=== Traitement de la page 105/136 ===
  - Traduction des blocks 1 à 16 sur 16
  ✅ Page 105 terminée (16 blocks traduits)

=== Traitement de la page 106/136 ===
  - Traduction des blocks 1 à 13 sur 13
  ✅ Page 106 terminée (13 blocks traduits)

=== Traitement de la page 107/136 ===
  - Traduction des blocks 1 à 15 sur 15
  ✅ Page 107 terminée (15 blocks traduits)

=== Traitement de la page 108/136 ===
  - Traduction des blocks 1 à 15 sur 15
  ✅ Page 108 terminée (15 blocks traduits)

=== Traitement de la page 109/136 ===
  - Traduction des blocks 1 à 16 sur 19
  - Traduction des blocks 17 à 19 sur 19
  ✅ Page 109 terminée (19 blocks traduits)

=== Traitement de la page 110/136 ===
  - Traduction des blocks 1 à 16 sur 19
  - Traduction des blocks 17 à 19 sur 19
  ✅ Page 110 terminée (

In [6]:
sizes = (
    list(range(100, 49, -10)) +
    list(range(50, 19, -5)) +
    list(range(20, 13, -1)) +
    [round(x, 1) for x in [14 - i*0.1 for i in range(int((14-4)/0.1)+1)]]
)

In [8]:
for page_index, page in enumerate(doc):
    blocks = translated_pages[page_index]
    for coords, text in blocks:
        rect = fitz.Rect(coords)

        # dessiner un rectangle jaune
        page.draw_rect(rect, color=(1, 1, 0), fill=(1, 1, 0), fill_opacity=0.9, overlay=True)

        # --- ajustement automatique ---
        # On part d'une taille max et on réduit jusqu'à ce que ça rentre
        index_size = 0
        font_size = sizes[index_size]

        while index_size < len(sizes)-1 :  # limite basse
            rc = page.insert_textbox(
                rect,
                text,
                fontsize=font_size,
                color=(0, 0, 0),
                align=3,
                render_mode=3  # "mesure seulement", sans dessiner
            )
            if rc >= 0:  # le texte tient dans le rectangle
                break
            index_size += 1
            font_size = sizes[index_size]

        # Insérer réellement avec la bonne taille
        page.insert_textbox(
            rect,
            text,
            fontsize=font_size,
            color=(0, 0, 0),
            align=3
        )

OUTPUT_PDF = f"{SOURCE_PDF}_FR.pdf"
doc.save(OUTPUT_PDF)
print("PDF traduit sauvegardé sous", OUTPUT_PDF)

PDF traduit sauvegardé sous /Users/gauthier/Documents/Travail/Projets/IFG/TASK HUB/Le cadre théorique/04 Sources essentielles de TASK™/9789240074323-eng.pdf_FR.pdf
